# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

In [ ]:
df = pd.read_csv('/datasets/autos.csv')
display(df.head())
df.info()
df.hist(figsize=(10,15));

Файл содержит данные по 354369 технических характеристик и цен других автомобилей. Каждая строка данных описывается 16 признаками (7 количественных и 9 строчных).

В признаках VehicleTypeб Gearbox, Model, FuelType, Repaired есть пропущенные значения. Их необходимо обработать.

Также необходимо проверить данные на дублиткаты.

В файле есть свойства, которые не только не важны для обучения модели, но и могут послужить причиной ошибок, если их учиывать: дата скачивания анкеты из базы, дата создания анкеты, количесвто картинок, почтовый индекс и дата последней активности пользователя. Эти столбцы нужно удалить.

После удаления фамилии останутся 6 категориальных столбцов. Их необходимо закодировать с помощью OHE.

Кроме того, в подготовку данных должно войти разбиение выборки на обучающую, валидационную и тестовую. А также масштабирование численных признаков.

In [ ]:
#Удаление лишних столбцов 
data = df.drop(['DateCrawled','DateCreated','NumberOfPictures','PostalCode','LastSeen'], axis=1)

In [ ]:
#Подсчет пропусков в столбцах
data.isna().sum()

In [ ]:
#Просмотр уникальных значений в категориальных столбцах
print('VehicleType:', data['VehicleType'].unique())
print('Gearbox:', data['Gearbox'].unique())
print('Model:', data['Model'].sort_values().unique())
print('FuelType:', data['FuelType'].unique())
print('Repaired:', data['Repaired'].unique())

In [ ]:
data['Gearbox'].value_counts()

В столбцах VehicleType, Model и FuelType целесообразно заполнить пропущенные значения вариантом 'other', т.к. мы сами его не можем указать в виду недостаточности обоснований. 

Больше всего значений в столбце Gearbox принадлежат значению "manual", пропуски предлагается заполнить им же. 

Пропуски в столбце Repaired будут заполнены значением 'no'.

In [ ]:
#Заполнение пропусков выбранными раннее значениями
data['VehicleType'] = data['VehicleType'].fillna('other')
data['Model'] = data['Model'].fillna('other')
data['FuelType'] = data['FuelType'].fillna('other')
data['Gearbox'] = data['Gearbox'].fillna('manual')
data['Repaired'] = data['Repaired'].fillna('no')
data.isna().sum()

При просмотре диаграмм с численными методами, можно обнаружить аномалии, которые необходимо обработать.

In [ ]:
data['RegistrationYear'].hist(bins=100, figsize=(12,9))
data['RegistrationYear'].describe()

В годе регистрации наблюдаются выбросы. Узнаем максимальную дату скачивания анкеты автомобиля, чтобы убрать правые даты.

In [ ]:
date = pd.to_datetime(df['DateCrawled'])
date.max()

Таким образом, даты регистрации после 2016 года некорректны. Машины появились не раньше 1900 года. Это дает возможность убрать ненужные года.

In [ ]:
data = data.query('1899 < RegistrationYear < 2017')
data['RegistrationYear'].hist(bins=100, figsize=(12,9))
data['RegistrationYear'].describe()

По левой части гистограммы видно, что разумно использовать 1960 год как левую границу, так как это старые, но возможно еще работающие машины.

In [ ]:
data = data.query('1959 < RegistrationYear < 2017')

Рассмотрим выбросы в столбце Мощности

In [ ]:
data['Power'].hist(bins=100, figsize=(12,9))
data['Power'].describe()

In [ ]:
data = data.query('Power < 1001')
data['Power'].hist(bins=100, figsize=(12,9))
data['Power'].describe()

Из графика видно, что многие машины имеют мощность 0. Эти строки не подлежат восстановлению и будут удалены. После изучения материалов по машиннам, становится ясно, что минимальное количество лошадинных сил необходимых машине примрно равно 20. По правому отсечению возьмеп границу в 400 л.с.

In [ ]:
data = data.query('20 < Power < 401')
data['Power'].hist(bins=100, figsize=(12,9))
data['Power'].describe()

In [ ]:
data['Price'].value_counts()

В данных есть 10772 записи, где цена равна 0. Так быть не может, их необходимо убрать. 

In [ ]:
data = data.query('Price > 0')
data['Price'].hist(bins=100, figsize=(12,9))
data['Price'].describe()

Выделим нижнюю границу с помощью квантиля 5%.

In [ ]:
data['Price'].quantile([.05])

Стоимость 450 евро по текущему курсу примерно равно 36 000 рублей. Это минимальная цена для продажи поддержанного автомобиля. 

In [ ]:
data = data.query('450 <= Price')

**Вывод:** была произведена подготовка данных для дальнейшего обучения моделей.

## Обучение моделей

Для работы были выбраны модели LightGBM, DecisionTreeRegressor, CatBoost и LinearRegression.

### Разбиение данных на выборки

In [ ]:
features = data.drop(['Price'], axis=1)
target = data['Price']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

#Прямое кодирование
encoder = OrdinalEncoder()
cat_columns = features_train.select_dtypes(include='object').columns.to_list()
#print(cat_columns)
cat_features = features_train[cat_columns]
cat_features = pd.DataFrame(encoder.fit_transform(cat_features),
                                columns=cat_features.columns, 
                                index=cat_features.index)
features_train = features_train.copy()
for column in cat_columns:
    features_train[column] = cat_features[column]

cat_features = features_test[cat_columns]
cat_features = pd.DataFrame(encoder.transform(cat_features),
                                columns=cat_features.columns, 
                                index=cat_features.index)
features_test = features_test.copy()
for column in cat_columns:
    features_test[column] = cat_features[column]

ohe_features_train = pd.get_dummies(features_train, drop_first=True)
ohe_features_test = pd.get_dummies(features_test, drop_first=True)

print(features_train.shape)
print(features_test.shape)
print(ohe_features_train.shape)
print(ohe_features_test.shape)

### Обучение моделей

#### Линейная регрессия

In [ ]:
%%time

slr = LinearRegression()
pipeline = make_pipeline(StandardScaler(), slr)
cv_results = cross_validate(pipeline, ohe_features_train, target_train, cv=5, 
                         scoring='neg_root_mean_squared_error')

In [ ]:
final_score_slr = cv_results['test_score'].mean()
print('Final score:  {:.3f}'.format(final_score_slr))
fit_time_slr = cv_results['fit_time'].mean()
score_time_slr = cv_results['score_time'].mean()
print('время обучения',cv_results['fit_time'].mean())
print('время предсказания',cv_results['score_time'].mean())


#### Дерево решений

In [ ]:
%%time

#Дерево решений
dtr = DecisionTreeRegressor(random_state=12345)
parametrs = { 'min_samples_leaf': [1, 2, 3],
              'max_depth': [1,3,5]}
grid_dtr = GridSearchCV(dtr, parametrs, cv=5, scoring='neg_root_mean_squared_error', n_jobs=2)
grid_dtr.fit(features_train, target_train)

In [ ]:
print(grid_dtr.best_params_)
final_score_dtr = grid_dtr.best_score_
print(grid_dtr.best_score_)
fit_time_dtr = grid_dtr.cv_results_['mean_fit_time'].mean()
print('Сренее время обучения:',grid_dtr.cv_results_['mean_fit_time'].mean())
score_time_dtr = grid_dtr.cv_results_['mean_score_time'].mean()
print('Сренее время предсказания:',grid_dtr.cv_results_['mean_score_time'].mean())

#### CatBoost

In [ ]:
%%time


cbr = CatBoostRegressor(random_state=12345)
parametrs = { 'learning_rate':[0.1, 0.5, 0.8],
              'depth': [1,3,5]}
grid_cbr = GridSearchCV(cbr,parametrs, cv=5, scoring='neg_root_mean_squared_error', n_jobs=2)
grid_cbr.fit(features_train, target_train)

In [ ]:
print(grid_cbr.best_params_)
final_score_cbr = grid_cbr.best_score_
print(grid_cbr.best_score_)
fit_time_cbr = grid_cbr.cv_results_['mean_fit_time'].mean()
print('Сренее время обучения:',grid_cbr.cv_results_['mean_fit_time'].mean())
score_time_cbr = grid_cbr.cv_results_['mean_score_time'].mean()
print('Сренее время предсказания:',grid_cbr.cv_results_['mean_score_time'].mean())

#### LightGBM

In [ ]:
%%time

lgbm = lgb.LGBMRegressor(objective ='regression',metric = 'mae',random_state = 12345)
parametrs = {
    'num_leaves':[26,36] ,  
    'n_estimators':[500,1000]} 

grid_lgbm = GridSearchCV(lgbm,parametrs, cv=5, scoring='neg_root_mean_squared_error', n_jobs=2)
grid_lgbm.fit(features_train, target_train)

In [ ]:
print(grid_lgbm.best_params_)
final_score_lgbm = grid_lgbm.best_score_
print(grid_lgbm.best_score_)
fit_time_lgbm = grid_lgbm.cv_results_['mean_fit_time'].mean()
print('Сренее время обучения:',grid_lgbm.cv_results_['mean_fit_time'].mean())
score_time_lgbm = grid_lgbm.cv_results_['mean_score_time'].mean()
print('Сренее время предсказания:',grid_lgbm.cv_results_['mean_score_time'].mean())

### Анализ моделей

In [ ]:
index = ['LinearRegression',
         'DecisionTreeRegressor',
         'CatBoostRegressor',
         'LGBMRegressor']
data = {'RMSE на обучающей выборке, евро':['{:4.0f}'.format(final_score_slr * -1),
                            '{:4.0f}'.format(final_score_dtr * -1),
                            '{:4.0f}'.format(final_score_cbr * -1),
                            '{:4.0f}'.format(final_score_lgbm * -1)],
        'Время обучения модели, сек':[fit_time_slr,
                                          fit_time_dtr,
                                          fit_time_cbr,
                                          fit_time_lgbm],
        'Время предсказания модели, сек':[score_time_slr,
                                      score_time_dtr,
                                      score_time_cbr,
                                      score_time_lgbm],
        }

scores_data = pd.DataFrame(data=data, index=index)
scores_data

Из таблицы видно, что Линейная регрессия не достигла необходимого минимума в метрике. У CatBoost хорошая метрика, но самое большое время обучения. Из оставшихся двух выберем LGBM, т.к. у нее лучше метрика, и не очень большое время обучения и предсказания.

**Вывод:** были подобраны лучшие параметры для моделей LightGBM, DesicinTreeRegressor, CatBoost и LinearRegression, они были обучены на лучших параметрах, также с помощью них были получены предсказания; было измерено время их обучения и предсказания; получены метрики в каждом случае.

## Тестирование лучшей модели

In [ ]:
best_model = lgb.LGBMRegressor(objective ='regression',metric = 'mae',random_state = 12345, n_estimators=1000, num_leaves=36)
best_model.fit(features_train, target_train)
predictions = best_model.predict(features_test)
print('RMSE для тестовой выборке',mean_squared_error(target_test, predictions)**0.5)

In [ ]:
model_dummy_regressor = DummyRegressor()
model_dummy_regressor.fit(features_train, target_train)
predictions_test_dummy = model_dummy_regressor.predict(features_test)
print("Тестовая выборка для сырья:", mean_squared_error(target_test, predictions_test_dummy)**0.5)

**Вывод:** Лучшая модель CatBoost была проестирована на тестовой выборке.

Модель проверена на адекватность. Модель показала меньшие показатели RMSE, чем простейшая.

## Вывод

Данная работа выполнялась для сервиса по продаже автомобилей с пробегом «Не бит, не крашен». разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. Было необходимо построить модель для определения стоимости, учитывая качество предсказания, скорость предсказания и время обучения.

Файл содержит данные по 354369 историческим данным: технические характеристики, комплектации и цены автомобилей. Каждая строка данных описывается 16 признаками (7 количественных и 9 строчных). 

В признаках VehicleType, Gearbox, Model, FuelType, Repaired были пропущенные значения. Они были обработаны. В столбцах VehicleType, Model и FuelType пропущенные значения были заполнены вариантом 'other'. Больше всего значений в столбце Gearbox содержат значение "manual", пропуски заполнили им же. Пропуски в столбце Repaired заполнены значением 'no'.

В файле содержались свойства, которые не только не важны для обучения модели, но и могли послужить причиной ошибок, если их учиывать: дата скачивания анкеты из базы, дата создания анкеты, количесвто картинок, почтовый индекс и дата последней активности пользователя. Эти столбцы были удалены.

При просмотре диаграмм с численными методами, были обнаружены аномалии в столбцах год регистрации и мощность, которые были обработаны. 

После удаления остались 6 категориальных столбцов. Они были закодированы с помощью OHE для линейной модели. Для остальных моделей было использовно порядковое кодирование.

Данные были разбиты на обучающую и тестовую выборки.


Были подобраны лучшие параметры для моделей LightGBM, DesicinTreeRegressor, CatBoost и LinearRegression, они были обучены на лучших параметрах, также с помощью них были получены предсказания; было измерено время их обучения и предсказания; получены метрики в каждом случае.

Из таблицы выше видно, что Линейная регрессия не достигла необходимого минимума в метрике. У CatBoost хорошая метрика, но самое большое время обучения. Из оставшихся двух была выбрана LGBM, т.к. у нее лучше метрика, и не очень большое время обучения и предсказания.